In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import CallbackAny2Vec
import ast
from tqdm import tqdm
from joblib import Parallel, delayed
import datetime
import gc
import logging

logging.basicConfig(filename='process_errors.log', level=logging.ERROR, format='%(asctime)s %(levelname)s %(message)s')

c:\Users\loopi\anaconda3\envs\week4\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
open_path = './daum_data/01. preprocessed_daum_data.csv'
preprocessed_df = pd.read_csv(open_path)

In [3]:
preprocessed_df['date'] = pd.to_datetime(preprocessed_df['date'], format = '%Y%m%d')

set_preiod = 1 # 날짜 필터 기간 설정, 0 일 경우 데이터 전체
# 최근 날짜부터 설정 달까지 데이터프레임 자르기
if set_preiod:
    recent_date = preprocessed_df['date'].max()
    date_preoid = recent_date - pd.DateOffset(months=set_preiod)
    pre_copy_df = preprocessed_df[(preprocessed_df['date'] >= date_preoid) & (preprocessed_df['date'] <= recent_date)]
else:
    pre_copy_df = preprocessed_df.copy()
    
print(date_preoid, recent_date)

2024-06-30 00:00:00 2024-07-31 00:00:00


In [4]:
# 메모리 정리
del preprocessed_df
gc.collect()

0

In [5]:
pre_copy_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66610 entries, 30133 to 760607
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   tag                   66610 non-null  object        
 1   date                  66610 non-null  datetime64[ns]
 2   article_url           66610 non-null  object        
 3   letter_ctn            66610 non-null  int64         
 4   word_ctn              66610 non-null  int64         
 5   sentence_ctn          66610 non-null  int64         
 6   clear_sentence_split  66610 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 4.1+ MB


In [6]:
# 문자열로 저장된 데이터를 실제 리스트로 변환
def convert_to_list(x):
    if isinstance(x, str):  # 데이터가 문자열일 경우만 변환
        return ast.literal_eval(x)
    return x  # 이미 리스트인 경우 변환하지 않음

# 값 리스트화
pre_copy_df['clear_sentence_split'] = pre_copy_df['clear_sentence_split'].apply(convert_to_list)


In [7]:
# 문장 처리 함수 (각 프로세스에서 동작)
def process_sentences(row):
    okt = Okt()
    tagged_data_local = []
    sentences = row['clear_sentence_split']
    sent_id = row['tag']
    
    try:
        for sentence in sentences:
            if sentence.strip():  # 빈 문장 방지
                words = okt.morphs(sentence)
                tagged_data_local.append(TaggedDocument(words=words, tags=[sent_id]))
    except Exception as e:
        logging.error(f"Error tag: {sent_id}: {e}")
        
    return tagged_data_local
        
# 진행률 표시를 위한 함수
def process_with_progress(df):
    results = Parallel(n_jobs=4)(delayed(process_sentences)(row[1]) for row in tqdm(df.iterrows(), total=len(df)))  # n_job cpu 코어 수
    return [doc for result in results for doc in result]

if __name__ == '__main__':

    # 진행률 표시 + 병렬 처리
    tagged_data = process_with_progress(pre_copy_df)

    print('Tokenizer finished.')

100%|██████████| 66610/66610 [11:18<00:00, 98.14it/s] 


Tokenizer finished.


In [8]:
# 멀티프로세스 print
class EpochLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0
    
    def on_epoch_start(self, model):
        print(f'Start epoch {self.epoch}')
    
    def on_epoch_end(self, model):
        print(f'Finished epoch {self.epoch}')
        self.epoch += 1

In [10]:
# Doc2Vec 모델 초기화 (병렬 처리 활성화)
model = Doc2Vec(
    vector_size=100,     # 벡터 크기 (최적화를 위해 크기를 줄임)
    window=5,           # 컨텍스트 윈도우 크기
    min_count=2,        # 최소 단어 빈도 (빈도수가 낮은 단어 무시)
    workers=8,          # 사용하고자 하는 CPU 스레드 수 (최대 CPU 코어 사용)
    epochs=5,           # 에포크 수 
    dm=1                # Distributed Memory 모델 사용
)

print('start Doc2vec modeling.')
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs, callbacks=[EpochLogger()])   # 완료시 log 표시

model.save("./daum_data/doc2vec_daum_model.model")

print(f'make model complite. all lenght : {len(tagged_data)}')

start Doc2vec modeling.
Finished epoch 0
Finished epoch 1
Finished epoch 2
Finished epoch 3
Finished epoch 4
make model complite. all lenght : 731511
